In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("w0kfPhGau5c")



# [Pandas Data Wrangling](http://pandas.pydata.org/)

In [ ]:
% matplotlib inline

In [ ]:
import os
import sqlite3 as sqlite
DATADIR = os.path.join("..", "Resources")
os.path.exists(DATADIR)
print(DATADIR)

In [ ]:
import pandas as pd

In [ ]:
%matplotlib inline
import numpy as np

## Reading in NaN/Missing Values, etc.

## Modifying Values 
### ``replace()``


In [ ]:
elevation = pd.read_table(os.path.join(DATADIR,"elevation.txt"))

In [ ]:
elevation

This data is easy to read, but had to compute with. First off, we have numeric values except for "Sea level". We can use ``replace`` to replace "Sea level" with "0 feet" to match the other cells

In [ ]:
elevation.replace("Sea level","0 feet")


We can also use regular expressions to identify and change data.

In [ ]:
import re
r2 = re.compile("feet")

In [ ]:
elevation.replace("Sea level","0 feet").replace(r2,"").to_csv(os.path.join(DATADIR,
                                                                           "elevation2.txt"),
                                                             sep="\t")
elevation2=elevation.replace("Sea level","0 feet").replace(r2,"")
elevation2

## What is the data type of the elevation cells

In [ ]:
print(elevation2["Highest elevation"][0])
print(type(elevation2["Highest elevation"][0]))


## Can we convert elevations to numeric values

In [ ]:
help(elevation2.convert_objects)

In [ ]:
elevation2.convert_objects(convert_numeric=True)

### Pandas didn't know how to deal with "," in numbers

* Two approaches
    1. When reading in numeric values we can specify the ``thousands`` keywoard
    2. We can use the [``locale``](https://docs.python.org/3.5/library/locale.html#module-locale) package
* I saved the dataframe where we had replaced ``Sea level`` and ``feet``

In [ ]:
pd.read_table(os.path.join(DATADIR,"elevation2.txt"),
                           thousands=",",
                           index_col='State')

In [ ]:
import locale
print(locale.getlocale())


In [ ]:
help(locale.setlocale)

In [ ]:
elevation2.head()

In [ ]:
locale.setlocale(locale.LC_NUMERIC, '') # I'm a little confused by this
elevation2['Lowest elevation'] = \
elevation2.apply(lambda row: locale.atof(row['Lowest elevation']),
                     axis=1)
elevation2['Highest elevation'] = \
elevation2.apply(lambda row: locale.atof(row['Highest elevation']),
                     axis=1)
elevation2['Average elevation'] = \
elevation2.apply(lambda row: locale.atof(row['Average elevation']),
                     axis=0)
elevation2

In [ ]:
conversion = locale.localeconv()
locale.getlocale()

## Creating a new column

In [ ]:
elevation2['Range elevation'] = \
elevation2.apply(lambda row: row['Highest elevation'] - 
                             row['Lowest elevation'],axis=1 )
elevation2.head()

In [ ]:
elevation2['Range elevation the smart way'] = \
   elevation2["Highest elevation"] - elevation2["Lowest elevation"]
elevation2.head()

## Dropping Rows based on values

In [ ]:
elevation2['Highest elevation'] > 8000

In [ ]:
elevation2[(elevation2['Highest elevation'] > 8000) & (elevation2['Average elevation'] > 2000)]

In [ ]:
pd.read_table(os.path.join(DATADIR,"mimic2_radreports_100.txt")).shape

### ``dropna()``

``dropna()`` allows us to drop rows and columns that have **any** or **all** NaN values

In [ ]:
pd.read_table(
        os.path.join(DATADIR,
                     "mimic2_radreports_100.txt")).dropna(how="any", axis=1).shape

In [ ]:
reports = pd.read_table(
        os.path.join(DATADIR,
                     "mimic2_radreports_100.txt"),
                       usecols=["subject_id", "charttime", "text"]).dropna(how="any", axis=1)

In [ ]:
reports.shape

In [ ]:
reports.iloc[0]["text"]